In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import joblib

In [2]:
data=pd.read_csv('/home/thiri/Documents/rt_model/data/nov_april_prepared_data.csv')

model = joblib.load('/home/thiri/Documents/rt_model/saved_model/xgboost_model.pkl') #Change models as needed

/usr/lib/python3.12/pickle.py:1759: UserWarning: [14:11:28] WARNING: /workspace/src/collective/../data/../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


### **load actual May terminated/active data**

In [3]:
real_may_terminated=pd.read_csv('/home/thiri/Documents/rt_model/actual_may_terminated_active_data/id_list_terminated_in_May.csv')
real_may_active=pd.read_csv('/home/thiri/Documents/rt_model/actual_may_terminated_active_data/id_list_active_in_May.csv')

### **Filtering 'data'** 

In [4]:
real_status_data=pd.concat([real_may_terminated, real_may_active])

In [5]:
real_status_data.shape

(121169, 3)

In [6]:
real_status_data['Customer ID'].nunique()

121169

In [7]:
df=data[data['Customer ID'].isin(real_status_data['Customer ID'])]

In [8]:
df.shape

(120593, 14)

In [9]:
priority_map = {'Low': 0, 'Medium': 1, 'High': 2}
df['Last Priority'] = df['Last Priority'].map(priority_map)

df['device_growth_ratio'] = df['avg_device_count_last_2_months'] / (df['overall_avg_device_count'] + 1e-5)

df['avg_usage_per_device'] = df['Average Data Usage'] / (df['overall_avg_device_count'] + 1e-5)

/tmp/ipykernel_349628/3369433123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Last Priority'] = df['Last Priority'].map(priority_map)
/tmp/ipykernel_349628/3369433123.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['device_growth_ratio'] = df['avg_device_count_last_2_months'] / (df['overall_avg_device_count'] + 1e-5)
/tmp/ipykernel_349628/3369433123.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [10]:
X_new = df.loc[:, model.feature_names_in_]
y_probs = model.predict_proba(X_new)[:, 1]

# Apply binary threshold classification (e.g., 0.4, 0.5)
threshold = 0.5
predicted_status = np.where(y_probs >= threshold, 'TERMINATED', 'ACTIVE')

df['Predicted_Status'] = predicted_status

predicted_data=df[['Customer ID', 'Predicted_Status']]

/tmp/ipykernel_349628/3489120383.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Predicted_Status'] = predicted_status


In [11]:
predicted_data

,Customer ID,Predicted_Status
0,2275947,ACTIVE
1,2150547,ACTIVE
2,2178246,TERMINATED
3,2347542,ACTIVE
4,2141477,TERMINATED
...,...,...
164338,2409654,TERMINATED
164339,2406087,TERMINATED
164351,2255050,TERMINATED
164370,2459519,TERMINATED


In [12]:
predicted_data['Predicted_Status'].value_counts()

Predicted_Status
ACTIVE        96666
TERMINATED    23927
Name: count, dtype: int64

### Checking

In [13]:
predicted_terminated = predicted_data[predicted_data['Predicted_Status']=='TERMINATED']
predicted_active = predicted_data[predicted_data['Predicted_Status']=='ACTIVE']

In [14]:
correct_terminated_count=real_may_terminated[real_may_terminated['Customer ID'].isin(predicted_terminated['Customer ID'])]

In [15]:
correct_terminated_count.shape

(939, 3)

In [16]:
correct_active_count=real_may_active[real_may_active['Customer ID'].isin(predicted_active['Customer ID'])]

In [17]:
correct_active_count.shape

(95950, 3)